You can download the `requirements.txt` for this course from the workspace of this lab. `File --> Open...`

# L2: Create Agents to Research and Write an Article

In this lesson, you will be introduced to the foundational concepts of multi-agent systems and get an overview of the crewAI framework.

The libraries are already installed in the classroom. If you're running this notebook on your own machine, you can install the following:
```Python
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29
```

In [31]:
!pip install --upgrade crewai crewai_tools langchain_community
!pip install -U openai


Python(61086) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 5.5 MB/s eta 0:00:005.6 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 545.9/545.9 kB 6.3 MB/s eta 0:00:00
  Attempting uninstall: litellm
    Found existing installation: litellm 1.59.8
    Uninstalling litellm-1.59.8:
      Successfully uninstalled litellm-1.59.8
  Attempting uninstall: crewai
    Found existing installation: crewai 0.100.1
    Uninstalling crewai-0.100.1:
      Successfully uninstalled crewai-0.100.1
  Attempting uninstall: crewai_tools
    Found existing installation: crewai-tools 0.33.0
    Uninstalling crewai-tools-0.33.0:
      Successfully uninstalled crewai-tools-0.33.0


Python(61111) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


  Attempting uninstall: openai
    Found existing installation: openai 1.61.1
    Uninstalling openai-1.61.1:
      Successfully uninstalled openai-1.61.1


In [32]:
from langchain_openai import ChatOpenAI

In [33]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

- Import from the crewAI libray.

In [34]:
from crewai import Agent, Task, Crew
from crewai_tools import SerperDevTool

- As a LLM for your agents, you'll be using OpenAI's `gpt-4o-mini`.

**Optional Note:** crewAI also allow other popular models to be used as a LLM for your Agents. You can see some of the examples at the [bottom of the notebook](#1).

In [ ]:
import os
from google.colab import userdata
openai_api_key = userdata.get("OPENAI_API_KEY")
grok_api_key = userdata.get("GROQ_API_KEY")
os.environ["OPENAI_API_KEY"] = openai_api_key
os.environ["OPENAI_MODEL_NAME"] = 'gpt-4o-mini'

In [35]:
from crewai import Agent, Crew, Process, Task
from dotenv import load_dotenv
load_dotenv()

# Ensure your SerperDev API key is set in the environment variables
os.environ["SERPER_API_KEY"] = os.environ["SERPER_DEV_API_KEY"]

llm_ollama= ChatOpenAI(
    model_name="llama3.2:3b",
    base_url="http://localhost:11434/v1",
)

llm_openai = ChatOpenAI(
    model_name="gpt-4o-mini",
    api_key=os.getenv("OPENAI_API_KEY"),
)

llm_groq = ChatOpenAI(
    model_name="llama3-70b-8192",
    base_url="https://api.groq.com/openai/v1",
    api_key=os.getenv("GROQ_API_KEY"),
)

llm  = llm_openai

## Creating Agents

- Define your Agents, and provide them a `role`, `goal` and `backstory`.
- It has been seen that LLMs perform better when they are role playing.

### Agent: Planner

**Note**: The benefit of using _multiple strings_ :
```Python
varname = "line 1 of text"
          "line 2 of text"
```

versus the _triple quote docstring_:
```Python
varname = """line 1 of text
             line 2 of text
          """
```
is that it can avoid adding those whitespaces and newline characters, making it better formatted to be passed to the LLM.

In [36]:
planner = Agent(
    role="Content Planner",
    goal="Plan engaging and factually accurate content on {topic} using internet search",
    backstory="You're working on planning a blog article "
              "about the topic: {topic}."
              "You collect information that helps the "
              "audience learn something "
              "and make informed decisions. "
              "Your work is the basis for "
              "the Content Writer to write an article on this topic."
              "You can search the internet for latest information using the serper dev tool provdied to you.",
    allow_delegation=False,
    llm=llm,
    tools = [SerperDevTool()],
	verbose=True,
    max_retry_limit=3
)

### Agent: Writer

In [37]:
writer = Agent(
    role="Content Writer",
    goal="Write insightful and factually accurate "
         "opinion piece about the topic: {topic} using internet search",
    backstory="You're working on a writing "
              "a new opinion piece about the topic: {topic}. "
              "You base your writing on the work of "
              "the Content Planner, who provides an outline "
              "and relevant context about the topic. "
              "You follow the main objectives and "
              "direction of the outline, "
              "as provide by the Content Planner. "
              "You also provide objective and impartial insights "
              "and back them up with information "
              "provide by the Content Planner. "
              "You acknowledge in your opinion piece "
              "when your statements are opinions "
              "as opposed to objective statements."
              "You can search the internet for latest information using the serper dev tool.",
    allow_delegation=False,
    llm = llm,
    tools=[SerperDevTool()],
    verbose=True,
    max_retry_limit=5
)

### Agent: Editor

In [38]:
editor = Agent(
    role="Editor",
    goal="Edit a given blog post to align with "
         "the writing style of the organization. ",
    backstory="You are an editor who receives a blog post "
              "from the Content Writer. "
              "Your goal is to review the blog post "
              "to ensure that it follows journalistic best practices,"
              "provides balanced viewpoints "
              "when providing opinions or assertions, "
              "and also avoids major controversial topics "
              "or opinions when possible.",
    allow_delegation=False,
    llm = llm,
    verbose=True
)

## Creating Tasks

- Define your Tasks, and provide them a `description`, `expected_output` and `agent`.

### Task: Plan

In [39]:
plan = Task(
    description=(
        "1. Prioritize the latest trends, key players, "
            "and noteworthy news on {topic}.\n"
        "2. Identify the target audience, considering "
            "their interests and pain points.\n"
        "3. Develop a detailed content outline including "
            "an introduction, key points, and a call to action.\n"
        "4. Include SEO keywords and relevant data or sources."
    ),
    expected_output="A comprehensive content plan document "
        "with an outline, audience analysis, "
        "SEO keywords, and resources.",
    agent=planner,
    llm=llm,
)

### Task: Write

In [40]:
write = Task(
    description=(
        "1. Use the content plan to craft a compelling "
            "blog post on {topic}.\n"
        "2. Incorporate SEO keywords naturally.\n"
		"3. Sections/Subtitles are properly named "
            "in an engaging manner.\n"
        "4. Ensure the post is structured with an "
            "engaging introduction, insightful body, "
            "and a summarizing conclusion.\n"
        "5. Proofread for grammatical errors and "
            "alignment with the brand's voice.\n"
    ),
    expected_output="A well-written blog post "
        "in markdown format, ready for publication, "
        "each section should have 2 or 3 paragraphs.",
    agent=writer,
    llm=llm,
)

### Task: Edit

In [41]:
edit = Task(
    description=("Proofread the given blog post for "
                 "grammatical errors and "
                 "alignment with the brand's voice."),
    expected_output="A well-written blog post in markdown format, "
                    "ready for publication, "
                    "each section should have 2 or 3 paragraphs.",
    agent=editor,
    llm=llm,
)

## Creating the Crew

- Create your crew of Agents
- Pass the tasks to be performed by those agents.
    - **Note**: *For this simple example*, the tasks will be performed sequentially (i.e they are dependent on each other), so the _order_ of the task in the list _matters_.
- `verbose=2` allows you to see all the logs of the execution.

In [46]:
crew = Crew(
    agents=[planner, writer, editor],
    tasks=[plan, write, edit],
    verbose=0
)

2025-02-13 07:46:08,327 - 7955664896 - __init__.py-__init__:537 - WARNING: Overriding of current TracerProvider is not allowed


## Running the Crew

**Note**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

In [47]:
result = crew.kickoff(inputs={"topic": "Elon musk is virtual president today."})

# Agent: Content Planner
## Task: 1. Prioritize the latest trends, key players, and noteworthy news on Elon musk is virtual president today..
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. Include SEO keywords and relevant data or sources.


# Agent: Content Planner
## Thought: I need to gather the latest information regarding Elon Musk being a "virtual president" to understand current trends, key players, and noteworthy news. This will help in developing a comprehensive content plan.
## Using tool: Search the internet with Serper
## Tool Input: 
"{\"search_query\": \"Elon Musk virtual president news 2023\"}"
## Tool Output: 
{'searchParameters': {'q': 'Elon Musk virtual president news 2023', 'type': 'search', 'num': 10, 'engine': 'google'}, 'organic': [{'title': "Musk has been empowered to be 'virtual prime minister' - YouTube", 'link': 'https://www.yout

- Display the results of your execution as markdown in the notebook.

In [48]:
from IPython.display import Markdown
Markdown(result.raw)

# Content Plan for "Elon Musk is Virtual President Today"

## Audience Analysis
**Target Audience:**
1. **Tech Enthusiasts**: Interested in the intersection of technology and politics, particularly how influential figures like Musk shape policy.
2. **Political Analysts**: Seeking insights into the implications of Musk's role in government and how it affects governance.
3. **General Public**: Citizens curious about the current political climate and the influence of tech billionaires in government.

**Interests:**
- The influence of technology in politics.
- The role of billionaires in shaping public policy.
- Current political events involving Elon Musk.

**Pain Points:**
- Confusion about the extent of Musk's influence in government.
- Concerns regarding the implications of privatization of public sectors.
- The potential risks of a tech-centric governance model.

## Content Outline
### Introduction
- Briefly introduce Elon Musk and his recent rise to a powerful position within the U.S. government.
- Define the term "virtual president" in the context of Musk's influence.

### Key Points
1. **Musk's Role and Influence**
   - Overview of Musk's involvement with the Department of Government Efficiency (DOGE).
   - Highlight key moments such as his participation in Oval Office meetings.
   - Discuss the public perception of Musk as a "shadow president."

2. **Impact on Policy and Governance**
   - Analyze how Musk's business strategies (like cost-cutting) affect governmental operations.
   - Discuss the executive orders signed by Trump that empower Musk.

3. **Public Reaction and Criticism**
   - Examine reactions from political figures and the general public.
   - Include quotes from prominent critics, like California Congressman Robert Garcia.

4. **Future Implications**
   - Speculate on the long-term effects of Musk's involvement in politics.
   - Discuss potential scenarios regarding the relationship between technology and government.

### Conclusion
- Summarize the key points discussed.
- Pose questions to the audience about the implications of Musk’s role in government.

### Call to Action
- Encourage readers to share their thoughts on Musk’s influence in politics through comments or social media.
- Suggest following the blog for updates on this ongoing story.

## SEO Keywords
- Elon Musk virtual president
- Musk DOGE influence
- tech billionaires in politics
- government efficiency department
- Musk Trump relationship
- shadow presidency

## Resources
1. [Musk's Oval Office Stunt Puts His Shadow Presidency in Plain Sight](https://news.yahoo.com/musk-oval-office-stunt-puts-123422803.html)
2. [How did Elon Musk become so powerful in the Trump administration?](https://www.npr.org/2025/02/12/g-s1-48392/how-did-elon-musk-became-so-powerful-in-the-trump-administration)
3. [Trump is giving Elon Musk an unprecedented amount of power](https://abcnews.go.com/538/trump-giving-elon-musk-unprecedented-amount-power/story?id=118439637)
4. [Elon Musk defends his work from Oval Office as Trump signs EO to boost DOGE efforts](https://www.delawarepublic.org/2025-02-11/elon-musk-defends-his-work-from-oval-office-as-trump-signs-eo-to-boost-doge-efforts)

This content plan aims to provide an engaging and informative article that captures the current trends surrounding Elon Musk's role in government, appealing to a wide audience and driving engagement.
```